# Lab 03 - Route Lookups

Since ConfigSync file structure is compartmentalized it is important to understand how those components work when 
developing new code.  To recap, all interactions that function with Cisco Viptela Vmanage should be contained within the viptela.py file, all methods to deal with data retrieved from the vManage API should go in helpers.py, and tie it all together in config_sync.py. If you need to add configuration data, it can be added into the viptela.cfg file.

In this lab we will build a several functions to gather and format route lookups using the Python SDK and other python libraries.

## Base Configuration
As in the previous labs, the base configuration we will start with logging, argparse, base classes for both ConfigSync(config_sync.py) and ViptelaClient (viptela.py).  The login function has already been defined, and called when ConfigSync is initialized.  Everything else will be added in the code segments below.

The files in the directory /Labs/lab03 have been set to a base configuration state described above.

**config_sync.py**

```python
from viptela import ViptelaClient
import argparse
import logging
import yaml
import helpers


def parse_arguments():
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", "-c", help="Path to the configuration file", default="viptela.cfg")
    parser.add_argument("--debug", "-d", help="Display debug logs", action="store_true")
    return parser.parse_args()


def init_logger(log_level=logging.INFO):
    logger = logging.getLogger(__file__)
    logger.setLevel(log_level)

    console_handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s %(levelname)s: %(message)s')
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    if log_level == 10:
        # create file handler which logs even debug messages
        fh = logging.FileHandler('log_file.txt')
        formatter = logging.Formatter('%(asctime)s %(levelname)s: %(message)s')
        fh.setFormatter(formatter)
        fh.setLevel(logging.DEBUG)
        logger.addHandler(fh)
    return logger


class ConfigSync:

    def __init__(self, config, log):
        self.log = log
        self.config_file = config
        self.log.info('Initializing ConfigSync Class.')
        self.config = self._parse_config(config)
        self.viptela = self._init_viptela_client(self.config)
        self.log.debug('Finished initializing the configSync class.')

    def _parse_config(self, config_file):
        self.log.info('Parsing the configuration file.')
        with open(config_file, 'r') as f:
            config = yaml.safe_load(f)
        self.log.debug(f'The following parameters were received: {config}')
        return config

    def _init_viptela_client(self, config):
        self.log.info('Initializing ViptelaClient class.')
        host = config.get('viptela_host')
        username = config.get('viptela_username')
        password = config.get('viptela_password')
        port = config.get('viptela_port')
        self.log.debug(f'Username is {username} and password is {password}')
        viptela = ViptelaClient(host, username=username, password=password, port=port, log=self.log)
        self.log.info('Login to Viptela.')
        viptela.login_sdk()
        return viptela


if __name__ == "__main__":
    args = parse_arguments()

    if args.debug:
        log = init_logger(logging.DEBUG)
    else:
        log = init_logger()

    log.info(f'Viptela ConfigSync script has started.')
    # Instantiate ConfigSync which will launch connection to Viptela, setup logging, and
    # load configuration from config file.
    cs = ConfigSync(config=args.config, log=log)
    # Start custom scripting here.  Use the CS object to interact with functions above or pull modules
    # directly from the helper file.

```


**viptela.py**

```python
import requests
from vmanage.api.authentication import Authentication


class ViptelaClient:

    def __init__(self, host, port=443, username='admin', password='admin', log=None):
        self.host = host
        self.port = port
        self.username = username
        self.password = password
        self.log = log
        if not log:
            raise Exception('The logger should not be None.')

        self.cookie = None
        self.session = None
        self.base_headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
        }
        self.base_url = f'https://{self.host}:{self.port}/'

        requests.packages.urllib3.disable_warnings()

        self.log.debug('ViptelaClient class initialization finished.')

    def login_sdk(self):
        self.log.debug(f'Login to Viptela vManage with Python SDK at host {self.host}.')
        self.session = Authentication(host=self.host, user=self.username, password=self.password,
                                 port=self.port).login()
        self.cookie = self.session.cookies._cookies[self.host]["/"]["JSESSIONID"]
        self.log.debug(f'Viptela provided authentication cookie: {self.session.cookies._cookies[self.host]["/"]["JSESSIONID"]}.')
        

```

**helpers.py** - File is empty because we haven't created any helpers yet.

```
<EMPTY>
```

**viptela.cfg*** - Default Config
```
---
viptela_host: '198.18.133.200'
viptela_username: 'admin'
viptela_password: 'admin'
viptela_port: 8443

```

## More detail

Let's look in more detail at what is going on in the config_sync.py file.  Below is the snipped of code at the bottom config_sync.py that launches **everything** that happens in this script.

**config_sync.py**
```python
.
.
. <INFO OMITTED FOR BREVITY>
.
if __name__ == "__main__":
    args = parse_arguments()

    if args.debug:
        log = init_logger(logging.DEBUG)
    else:
        log = init_logger()

    log.info(f'Viptela ConfigSync script has started.')
    # Instantiate ConfigSync which will launch connection to Viptela, setup logging, and
    # load configuration from config file.
    cs = ConfigSync(config=args.config, log=log)
    # Start custom scripting here.  Use the CS object to interact with functions above or pull modules
    # directly from the helper file.
```

## Look up routes based on argument input
In this exercise you will be adding a workflow to gather route information from all vEdge devices. The route lookup workflow will pull and filter all route information, and a helper module will take all filtered route information and parse it into a table to display on CLI.  The viptela function will be housed within the ViptelaClient class within viptela.py, and the print table module will be in the helpers.py file.  You will then add the instructions to config_sync to tell it to gather all route information using the new Viptela class, filter the routes based on argument inputs, and then pass that data to the new helper class.

### Step 1 - Open Pycharm on Desktop

### Step 2 - Open ViptelaClient project

### Step 3 - Open file /Labs/lab03/config_sync.py, /Labs/lab03/viptela.py, and /Labs/lab03/helpers.py in Pycharm tabs

### Step 4 - Review API Reference for the Viptela SDK

Latest SDK-API Documents:
https://python-viptela.readthedocs.io/en/latest/

dCloud Lab API Documents from current vManage instance:
https://198.18.133.200:8443/apidocs/

We will be interacting with the MonitorNetwork and Device modules, in previous labs you used the Device module so lets review the MonitorNetwork module. In the SDK docs navigate to api --> Submodules --> api.monitor_network. Find get_ip_route_table, this is the module you will be interacting with.  We must instantiate a copy of this class using the parameters required in the blue box and then call the appropriate function, get_ip_route_table for the results.  This module accepts multiple parameters, but we will only be using one, system_ip.

### Step 5 - viptela.py - import the 'MonitorNetwork' and 'Devices' modules

To use this piece of the Python SDK, we first have to import the module into our viptela.py file.  Add the following as the last lines in the import statements at the top of the file.

```python
from vmanage.api.device import Device
from vmanage.api.monitor_network import MonitorNetwork
```

### Step 6 - viptela.py - create new modules in ViptelaClient class

Once the module is imported, we can begin to interact with the module within our ViptelaClass.  Go to the viptela.py file, and scroll to the bottom of the file, which should be right below 'def login_sdk(self):'.  We are going to add our new functions here which will be called form within config_sync.py. Copy the code snippets below into the file in Pycharm.

The function within viptela.py should do three things.

1. Instantiate the Device class we just imported using settings we already have because we instantiated ViptelaClient.

2. Send the 'type' of device to the get_device_list function in the Viptela SDK and save that data to a variable called device_list

3. Return device_list

```python
    def get_devices_list(self, type):
        '''
        Get devices uses the Viptela Device API to fetch the list of devices
        either vedges or controllers and returns them in a list of dictionaries.

        :param type: Either 'vedges' or 'controllers'
        :return:
            result (list): Device list
        '''

        # Instantiate new device object to use the Device library
        device = Device(self.session, self.host, port=self.port)
        # Request list of all devices by type specified and store in variable
        device_list = device.get_device_list(type)
        return device_list
```

1. Instantiate the MonitorNetwork class we just imported using settings we already have because we instantiated ViptelaClient.

2. Send the vEdge 'system_ip' to the get_ip_route_table function in the Viptela SDK and save that data to a variable called route_list

3. Return route_list

```python
    def get_route_table(self, system_ip):
        '''
        Get route table retrieves the routes from selected vedge for later manipulation.

        :param type: DeviceID: DeviceIP address
        :return:
            result (list): route_list
        '''

        # Instantiate new device object to use the Device library
        monitor = MonitorNetwork(self.session, self.host, port=self.port)
        # Request list of all devices by type specified and store in variable
        route_list = monitor.get_ip_route_table(system_ip)
        return route_list
```

### Step 7 - helpers.py - Import necessary modules to helpers.py

In order to format the output of the results we are going to use a module called 'tabulate'. The 'time' library with also be used. Import these libraries into helpers.py at the top of the file.

```python
import tabulate
import time
```

### Step 8 - helpers.py - Create new function in helpers.py to print the route tables to screen

Add the following code snippet into helpers.py.  The function will take the input route_lists and create a list for headers based on the fields we will be pulling out of the data set.  To populate the table, iterate through the route list passed over, and for each route gather the fields specified in the headers.

Once the table is populated with all route, we pass both the table and headers into the 'tabulate' library.  We use try/except blocks as error handling.  In the event there is a unicode error, a modified version of this grid will be populated.

```python
def print_route_list(route_list):

    headers = [
        'Host-Name',
        'System IP',
        'Destination Route',
        'Routing Instance',
        'Route Metric',
    ]

    table = []

    for route in route_list:
            row = [
                route['vdevice-host-name'],
                route['vdevice-name'],
                route['route-destination-prefix'],
                route['routing-instance-name'],
                route['route-metric'],
            ]
            table.append(row)

    try:
        print(tabulate.tabulate(table, headers, tablefmt="fancy_grid"))
        time.sleep(1)
    except UnicodeEncodeError:
        print(tabulate.tabulate(table, headers, tablefmt="grid"))
        time.sleep(1)
```

### Step 9 - config_sync.py - Call the new viptela function

The function has been defined in viptela.py but we need to update config_sync.py to launch that function. First, create a new function under ConfigSync class that will request the list of vedge devices and iterate through each vEdge device to collect and filter the routes.  Second, call that function from under the "if __name__ == '__main__'" section of the script.  To avoid any confusion between the function name in ConfigSync and the function name in ViptelaClient, prepend all ConfigSync functions with cs_.

1. Import the 'ipaddress' python library. Due to limitations in the vManage API we will have to use external libraries to filter our routes. The 'ipaddress' library will allow us to quickly check if an IP address is in a network given in CIDR format. At the top of the file add the following import statement.

```python
import ipaddress
```

2. Assume that we want to add this function to a collection of tools that we can run against our vManage instance. We are going to add another command line argument that will allow us to trigger this route lookup. Add the below code snippet under 'parse_arguments' function near the top of the script.

```python
parser.add_argument("--route", "-r", help="Display routes matching provided IPv4 Address", )
```

3. Create the function cs_route_lookup and gather vedges to iterate through them.  Return the data.

```python
    def cs_route_lookup(self, route):
        vedges = self.viptela.get_devices_list('vedges')
        print(f'~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Route lookup: {route} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        route_list = []
        for device in vedges:
            if 'host-name' in device and 'system-ip' in device:
                routes = self.viptela.get_route_table(device['system-ip'])
                for x in routes:
                    if ipaddress.ip_address(route) in ipaddress.ip_network(x['route-destination-prefix']):
                        route_list.append(x)
        return route_list
```

4. Call that function from under the "if __name__ == '__main__'".  An IF statement will be needed to check if the '-r' argument has been triggered. If it has been triggered it will contain a string with the IP address.

```python
    # Print Route information if requested
    if type(args.route) == str:
        helpers.print_route_list(cs.cs_route_lookup(args.route))
```


### Step 10 - Save & launch the code again

Now we should have everything needed to launch this function.  This will need to be run with the command line argument -r and an ip address.
Activate the virtual environment from the terminal.
1. Click on the terminal tab on the bottom of pycharm
2. You should be in the 'Viptela-ConfigSync' directory, enter the following in the terminal to activate the virtual env: .\venv\Scripts\activate.ps1
3. Change to the \Labs\lab03\ directory: cd .\Labs\lab03\
4. Run the script with the following command in the terminal: python .\config_sync.py -r 192.168.1.5
5. This will gather routes for the given IP address, due to some limitation of the API you will see overlapping routes.

### NOTE: If you have any issues compare to the files in the /final folder.